In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import torch
import numpy as np

In [3]:
from importlib import reload
import gedi_adapter
reload(gedi_adapter)
from gedi_adapter import GediAdapter

/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
t5name = 'SkolkovoInstitute/t5-paraphrase-paws-msrp-opinosis-paranmt'

In [5]:
import sys
sys.path.append(os.path.abspath('../transfer_utils/'))

import text_processing
reload(text_processing);

In [6]:
tokenizer = AutoTokenizer.from_pretrained(t5name)

/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
para = AutoModelForSeq2SeqLM.from_pretrained(t5name)
para.resize_token_embeddings(len(tokenizer)) 

/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/transformers/modeling_utils.py:399: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feat

Embedding(32100, 768)

In [8]:
model_path = 'SkolkovoInstitute/gpt2-base-gedi-detoxification'
gedi_dis = AutoModelForCausalLM.from_pretrained(model_path)

/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at SkolkovoInstitute/gpt2-base-gedi-detoxification were not used when initializing GPT2LMHeadModel: ['logit_scale', 'bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
NEW_POS = tokenizer.encode('normal', add_special_tokens=False)[0]
NEW_NEG = tokenizer.encode('toxic', add_special_tokens=False)[0]

In [10]:
# add gedi-specific parameters
if os.path.exists(model_path):
    w = torch.load(model_path + '/pytorch_model.bin', map_location='cpu')
    gedi_dis.bias = w['bias']
    gedi_dis.logit_scale = w['logit_scale']
    del w
else:
    gedi_dis.bias = torch.tensor([[ 0.08441592, -0.08441573]])
    gedi_dis.logit_scale = torch.tensor([[1.2701858]])
print(gedi_dis.bias, gedi_dis.logit_scale)

tensor([[ 0.0844, -0.0844]]) tensor([[1.2702]])


In [11]:
%%time

dadapter = GediAdapter(model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=5, target=1, neg_code=NEW_NEG, pos_code=NEW_POS, lb=None, ub=None)
text = 'The internal policy of Trump is flawed.'
print('====BEFORE====')
print(text)
print('====AFTER=====')
inputs = tokenizer.encode(text, return_tensors='pt').to(para.device)
result = dadapter.generate(inputs, do_sample=True, num_return_sequences=3, temperature=1.0, repetition_penalty=3.0, num_beams=1)
for r in result:
    print(tokenizer.decode(r, skip_special_tokens=True))

/media/gabriel/Datos Linux/loncos/Text-Detoxification-in-Spanish/env/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


====BEFORE====
The internal policy of Trump is flawed.
====AFTER=====
her internal dialogue is stupid.
he is an idiot.
the president’s asexual policy is flawed.
CPU times: user 21 s, sys: 13.9 ms, total: 21 s
Wall time: 3.52 s


In [12]:
import torch
device = torch.device('cuda:0')
# device = torch.device('cpu')

In [13]:
import gc

def cleanup():
    gc.collect()
    if torch.cuda.is_available() and device.type != 'cpu':
        with torch.cuda.device(device):
            torch.cuda.empty_cache()

In [14]:
para.to(device);
para.eval();

In [15]:
gedi_dis.to(device);
gedi_dis.bias = gedi_dis.bias.to(device)
gedi_dis.logit_scale = gedi_dis.logit_scale.to(device)
gedi_dis.eval();

In [17]:
with open('../../data/processed/toxicCorpusWSpellErr.txt', 'r') as f:
    test_data = [line.strip() for line in f.readlines()]
print(len(test_data))

142907


In [18]:
dadapter = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=10, target=0, neg_code=NEW_NEG, pos_code=NEW_POS, 
    reg_alpha=3e-5, ub=0.01
)

In [19]:
def paraphrase(text, n=None, max_length='auto', beams=2):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(dadapter.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    result = dadapter.generate(
        inputs, 
        num_return_sequences=n or 1, 
        do_sample=False, temperature=0.0, repetition_penalty=3.0, max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    texts = [text_processing.text_postprocess(t) for t in texts]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [23]:
paraphrase(test_data[:3])

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 5.94 MiB is free. Including non-PyTorch memory, this process has 3.80 GiB memory in use. Of the allocated memory 3.64 GiB is allocated by PyTorch, and 98.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Evaluate

In [24]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
clf_name = 'SkolkovoInstitute/roberta_toxicity_classifier_v1'
clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device);
clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
def predict_toxicity(texts):
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out

In [26]:
predict_toxicity(['hello world', 'hello aussie', 'hello fucking bitch'])

array([5.052518e-05, 8.788534e-05, 9.996809e-01], dtype=float32)

# The baseline

In [27]:
# reload(gedi_adapter)
from gedi_adapter import GediAdapter


adapter2 = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer, 
    gedi_logit_coef=10, 
    target=0, pos_code=NEW_POS, 
    neg_code=NEW_NEG,
    reg_alpha=3e-5,
    ub=0.01,
    untouchable_tokens=[0, 1],
)


def paraphrase(text, max_length='auto', beams=5, rerank=True):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(adapter2.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    attempts = beams
    out = adapter2.generate(
        inputs, 
        num_beams=beams,
        num_return_sequences=attempts, 
        do_sample=False, 
        temperature=1.0, 
        repetition_penalty=3.0, 
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        output_scores=True, 
        return_dict_in_generate=True,
    )
    results = [tokenizer.decode(r, skip_special_tokens=True) for r in out.sequences]

    if rerank:
        scores = predict_toxicity(results)
    
    results = [text_processing.text_postprocess(t) for t in results]
    out_texts = []
    for i in range(len(texts)):
        if rerank:
            idx = scores[(i*attempts):((i+1)*attempts)].argmin()
        else:
            idx = 0
        out_texts.append(results[i*attempts+idx])
    return out_texts

torch.manual_seed(0)
paraphrase(['fuck you!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=3)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 15.94 MiB is free. Including non-PyTorch memory, this process has 3.79 GiB memory in use. Of the allocated memory 3.66 GiB is allocated by PyTorch, and 63.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [28]:
paraphrase(['fuck you!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=3, rerank=False)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 15.94 MiB is free. Including non-PyTorch memory, this process has 3.79 GiB memory in use. Of the allocated memory 3.69 GiB is allocated by PyTorch, and 36.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [29]:
paraphrase(['fuck you!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=10, rerank=False)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 21.94 MiB is free. Including non-PyTorch memory, this process has 3.78 GiB memory in use. Of the allocated memory 3.63 GiB is allocated by PyTorch, and 91.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
paraphrase(['fuck you!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=10, rerank=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 5.94 MiB is free. Including non-PyTorch memory, this process has 3.80 GiB memory in use. Of the allocated memory 3.66 GiB is allocated by PyTorch, and 78.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
batch_size = 2

In [31]:
import os
from tqdm.auto import tqdm, trange
import pandas as pd
test_data = pd.read_csv('../../data/processed/test.txt', sep='\t', header=None, names=['text'])

cleanup()

lines = test_data[:10]


for i in trange(int(len(lines) / batch_size + 1)):
    if i % 10 == 0:
        cleanup()
    t = i * batch_size
    batch = [line.strip() for line in lines[t:(t+batch_size)]]
    if not batch:
        continue

    try:
        res = paraphrase(batch, max_length='auto', beams=10)
    except RuntimeError:
        print('runtime error for batch ', i)
        try:
            cleanup()
            res = [paraphrase([text], max_length='auto', beams=3)[0] for text in batch]
        except RuntimeError:
            print('runtime error for batch ', i, 'even with batch size 1')
            res = batch
            cleanup()
    for out in res:
        print(out)

 17%|█▋        | 1/6 [00:00<00:02,  1.89it/s]

Text message


 33%|███▎      | 2/6 [00:00<00:01,  2.34it/s]

Text message


 50%|█████     | 3/6 [00:01<00:01,  2.58it/s]

...

Text message


 83%|████████▎ | 5/6 [00:01<00:00,  2.77it/s]

Text message


100%|██████████| 6/6 [00:02<00:00,  2.66it/s]

Text message


In [33]:
res

['Text message']